In [1]:
import pandas as pd
import re
import nltk
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import scapy
from nltk.tokenize import word_tokenize
import numpy as np


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/hugoroxo/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_insta

ModuleNotFoundError: No module named 'scapy'

## Input

In [2]:
train = pd.read_csv('nlp-getting-started/train.csv')
test = pd.read_csv ('nlp-getting-started/test.csv')

train.head()
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [3]:
print('Training shape: {}'.format(train.shape))
print('Test shape: {}'.format(test.shape))

Training shape: (7613, 5)
Test shape: (3263, 4)


## Pré processing

### Character cleaning

In [4]:
def clean_text(text):
    # Remover URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remover @menções
    text = re.sub(r'@\w+', '', text)
    # Remover hashtags
    text = re.sub(r'#', '', text)
    # Remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    # Converter para minúsculas
    text = text.lower()
    return text


In [5]:
# Aplicar a função ao DataFrame
train['clean_text'] = train['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

# Exibir as primeiras linhas do DataFrame
print(test[['text', 'clean_text']].head())

                                                text  \
0                 Just happened a terrible car crash   
1  Heard about #earthquake is different cities, s...   
2  there is a forest fire at spot pond, geese are...   
3           Apocalypse lighting. #Spokane #wildfires   
4      Typhoon Soudelor kills 28 in China and Taiwan   

                                          clean_text  
0                 just happened a terrible car crash  
1  heard about earthquake is different cities sta...  
2  there is a forest fire at spot pond geese are ...  
3              apocalypse lighting spokane wildfires  
4      typhoon soudelor kills 28 in china and taiwan  


### Tokenization

In [6]:
# Função para tokenizar o texto
def tokenize_text(text):
    return word_tokenize(text)

In [7]:
# Aplicar a função ao DataFrame
train['tokens'] = train['clean_text'].apply(tokenize_text)
test['tokens'] = test['clean_text'].apply(tokenize_text)

# Exibir as primeiras linhas do DataFrame
print(train[['clean_text', 'tokens']].head())
print(test[['clean_text', 'tokens']].head())

                                          clean_text  \
0  our deeds are the reason of this earthquake ma...   
1              forest fire near la ronge sask canada   
2  all residents asked to shelter in place are be...   
3  13000 people receive wildfires evacuation orde...   
4  just got sent this photo from ruby alaska as s...   

                                              tokens  
0  [our, deeds, are, the, reason, of, this, earth...  
1      [forest, fire, near, la, ronge, sask, canada]  
2  [all, residents, asked, to, shelter, in, place...  
3  [13000, people, receive, wildfires, evacuation...  
4  [just, got, sent, this, photo, from, ruby, ala...  
                                          clean_text  \
0                 just happened a terrible car crash   
1  heard about earthquake is different cities sta...   
2  there is a forest fire at spot pond geese are ...   
3              apocalypse lighting spokane wildfires   
4      typhoon soudelor kills 28 in china and taiwan 

In [8]:
import nltk
nltk.download('stopwords')

nltk.data.path

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


['/Users/hugoroxo/nltk_data',
 '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data',
 '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data',
 '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [9]:
# Carregar o modelo de linguagem
nlp = spacy.load('en_core_web_sm')

# Função para normalizar o texto
def normalize_text(tokens):
    doc = nlp(' '.join(tokens))  # Juntar os tokens em uma string para processamento
    normalized = [token.lemma_ for token in doc if not token.is_stop]  # Lematização e remoção de stopwords
    return normalized

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/hugoroxo/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# Aplicar a função ao DataFrame
train['normalized_tokens'] = train['tokens'].apply(normalize_text)
test['normalized_tokens'] = test['tokens'].apply(normalize_text)

# Exibir as primeiras linhas do DataFrame
print(train[['tokens', 'normalized_tokens']].head())


                                              tokens  \
0  [our, deeds, are, the, reason, of, this, earth...   
1      [forest, fire, near, la, ronge, sask, canada]   
2  [all, residents, asked, to, shelter, in, place...   
3  [13000, people, receive, wildfires, evacuation...   
4  [just, got, sent, this, photo, from, ruby, ala...   

                                   normalized_tokens  
0         [deed, reason, earthquake, allah, forgive]  
1      [forest, fire, near, la, ronge, sask, canada]  
2  [resident, ask, shelter, place, notify, office...  
3  [13000, people, receive, wildfire, evacuation,...  
4  [get, send, photo, ruby, alaska, smoke, wildfi...  


In [ ]:
# Carregar o tokenizer e o modelo DistilBERT pré-treinado
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def generate_embeddings(dataframe, text_column):
    """
    Gera embeddings para um DataFrame usando DistilBERT.

    Parameters:
    - dataframe: DataFrame contendo os textos.
    - text_column: Nome da coluna que contém os textos.

    Returns:
    - embeddings_array: Array numpy contendo os embeddings gerados.
    """
    embeddings = []

    for tweet in dataframe[text_column]:
        # Tokenização
        inputs = tokenizer(tweet, return_tensors="pt", padding=True, truncation=True)

        # Obter os embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # Armazenar o vetor de embeddings
        embeddings.append(outputs.last_hidden_state.mean(dim=1).numpy())  # Média dos embeddings de todas as tokens

    # Converter a lista de embeddings para um array numpy
    embeddings_array = np.array(embeddings)

    return embeddings_array


In [ ]:
# Gerar embeddings para a base de dados de treinamento
train_embeddings_array = generate_embeddings(train, 'normalized_tokens')
train['embeddings'] = list(train_embeddings_array)

# Gerar embeddings para a base de dados de teste
test_embeddings_array = generate_embeddings(test, 'normalized_tokens')
test['embeddings'] = list(test_embeddings_array)


RuntimeError: Numpy is not available